In [1]:
%run imports.ipynb
%run utils.ipynb

from sklearn import preprocessing

In [2]:
data = pd.read_csv('bases/train.csv')

lines = [line.rstrip('\n') for line in open('bases/Reviews + Sentiment.txt','r',  encoding="utf8")]
data = data.assign(content_reviewSentiments = np.array(lines))

In [3]:
labels  = data['label']

mlb = MultiLabelBinarizer(classes=("Dep", "Per", "Sup", "Usa", "Mis"))
labelsdf = pd.DataFrame(mlb.fit_transform(labels), columns=['Dep', 'Per', 'Sup', 'Usa', 'Mis']) 
data = data.assign(label_dep=labelsdf['Dep'].values, label_per=labelsdf['Per'].values, label_sup=labelsdf['Sup'].values, label_usa=labelsdf['Usa'].values, label_mis=labelsdf['Mis'].values)
data.head(5)

,content_review,sentiment_analysis,domain_name,label,content_reviewSentiments,label_dep,label_per,label_sup,label_usa,label_mis
0,"I love the app, I have used it for years The o...",Positive,PhotosVideos,Usa,"I love the app, I have used it for years The o...",0,0,0,1,0
1,It would be better if they actually released t...,Positive,PhotosVideos,Mis,It would be better if they actually released t...,0,0,0,0,1
2,"Needs to be an option to pay monthly, or to op...",Negative,PhotosVideos,Mis,"Needs to be an option to pay monthly, or to op...",0,0,0,0,1
3,Please let there be a one week free trial I wo...,Positive,PhotosVideos,Mis,Please let there be a one week free trial I wo...,0,0,0,0,1
4,Can\'t connect to server. Failed big time. Ple...,Negative,PhotosVideos,Sup,Can\'t connect to server. Failed big time. Ple...,0,0,1,0,0


In [4]:
from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean_text(s):
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

In [5]:
#X = data['content_review'].apply(clean_text)
X = data['content_reviewSentiments'].apply(clean_text)
y = data.iloc[:, 5:9].values

In [6]:
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=42, test_size=0.2)
(X_tr.shape, y_tr.shape), (X_te.shape, y_te.shape)

(((4800,), (4800, 4)), ((1200,), (1200, 4)))

In [18]:
count_vectorizer = CountVectorizer()
count_train = count_vectorizer.fit_transform(X_tr) 
count_test = count_vectorizer.transform(X_te)

tfidf_vectorizer = TfidfVectorizer() 
tfidf_train = tfidf_vectorizer.fit_transform(X_tr) 
tfidf_test = tfidf_vectorizer.transform(X_te)

In [21]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from skmultilearn.problem_transform import ClassifierChain

CC = ClassifierChain(classifier=XGBClassifier(), require_dense=[False, True])

CC.fit(tfidf_train, y_tr)
y_pred = CC.predict(tfidf_test)

print("Acurácia = ", accuracy_score(y_te, y_pred))

Acurácia =  0.725


In [19]:
CC.fit(count_train, y_tr)
y_pred = CC.predict(count_test)

print("Acurácia = ", accuracy_score(y_te, y_pred))

Acurácia =  0.7008333333333333


In [30]:
print("Precision Score : ",precision_score(y_te, y_pred, average='micro'))
print("Recall Score : ",recall_score(y_te, y_pred, average='macro'))

Precision Score :  0.7068965517241379
Recall Score :  0.47018712729814116
